In [1]:
# increase data rate by opening the jupyter notebook with the following tail:
## 'jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10'
# Close and repoen the notebook if necessary

In [2]:
##!Before running this code, make sure to take all dataset out of 'Data' folder and place them to the right directory!

import re
import json
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook
import numpy as np

#create the list of dictinary formatted strings by extracting data item
#from twitter API scrape
#files = ['twtgun3.25.1.txt', 'twtgun3.25.2.txt','twtgun3.25.3.txt', 'twtgun3.25.4.txt']

group = []
for i in range(1,10):
   
    with open('twtgun3.25.' + str(i) + '.txt') as fhand:
        for item in fhand:
            dic = {}
            x = re.findall("^{'created_at': \'(.*?)\'", item)
            if len(x) > 0:
                dic["created_at"] = str(x[0])
            else:
                dic["created_at"] = None
            y = re.findall("{'full_text': \'(.*?)\'", item)
            if len(y) > 0:
                dic["full_text"] = str(y[0])
            else:
                dic["full_text"] = None
            z = re.findall(" 'text': \'(.*?)\'", item)
            if len(z) > 0:
                dic["text"] = str(z[0])
            else:
                dic["text"] = None
            a = re.findall("\'location\': \'(.*?)\'", item)
            if len(a) > 0:
                dic["location"] = str(a[0])
            else:
                dic["location"] = None
            b = re.findall("\'time_zone\': \'(.*?)\'", item)
            if len(b) > 0:
                dic["time_zone"] = str(b[0])
            else:
                dic["time_zone"] = None
            group.append(dic)

    fhand.close()

#convert the list into json object

with open('data.json2', 'w') as fp:
    json.dump(group, fp)

    # convert json file into pandas DataFrame
df = pd.read_json('data.json2')

In [3]:
df.sample(5)

,created_at,full_text,location,text,time_zone
376832,2018-03-25 18:25:11,##MarchForOurLives 🍰 LeTtrend Micro Scooter S...,None,RT @KateMata5469: ##MarchForOurLives 🍰 LeTtre...,Pacific Time (US & Canada)
311180,2018-03-25 17:39:19,None,"Atlanta, GA",RT @Phil_Lewis_: “Schoolhouse Glock” \n\n#Marc...,Eastern Time (US & Canada)
67097,2018-03-25 15:48:28,"This is ridiculous, @GOP should be ashamed... ...","Scarborough, ON","This is ridiculous, @GOP should be ashamed... ...",Quito
28875,2018-03-25 15:31:37,None,Los Angeles,RT @pattonoswalt: #MarchForOurLives https://t....,Pacific Time (US & Canada)
223068,2018-03-25 16:58:46,The Founding Fathers are spinning in their gra...,"Florida, USA",RT @johncardillo: The Founding Fathers are spi...,Eastern Time (US & Canada)


In [4]:
df2 = df.reindex(columns = ['created_at','location','time_zone','text','full_text'])
pd.options.display.max_rows=10
#print(df2)

In [5]:
df2['text'] = df2['text'].replace(r'\\n', '', regex=True).str.lower()
df2['text'] = df2['text'].str.strip()
df2['full_text'] = df2['full_text'].replace(r'\\n', '', regex=True).str.lower()
df2['full_text'] = df2['full_text'].str.strip()

In [6]:
print(df2.head())

           created_at          location                   time_zone  \
0 2018-03-25 09:12:15    Planet Awesome  Pacific Time (US & Canada)   
1 2018-03-25 09:12:22  Huarte (Navarra)                      Madrid   
2 2018-03-25 09:12:26      Portland, OR  Eastern Time (US & Canada)   
3 2018-03-25 09:12:27     Melbourne, FL                    Brisbane   
4 2018-03-25 09:12:28        Manchester                      London   

                                                text  \
0  rt @nickjonas: so inspired by the #marchforour...   
1  rt @almuariza: -“somos niños. no queremos que ...   
2  rt @davidhogg111: you guys rock we can and wil...   
3  rt @lisamareeloring: @proudresister your march...   
4  rt @imamofpeace: dear @davidhogg111, you’re cl...   

                                           full_text  
0                                               None  
1  -“somos niños. no queremos que nos maten”. los...  
2  my view before speaking at the south bay #marc...  
3  @proudresiste

In [7]:
#df2['full_text'].str.findall('\\n')

In [8]:
#make copy of df2 and name it df3
df3 = df2

In [9]:
#Take 'full_text' when available, otherwise take 'text'
df3['tweet'] = df3['full_text'].where(pd.notna(df3['full_text']), other = df3['text'])
#Use time_zone for proxy of location when location data is missing
df3['location_'] = df3['location'].where(pd.notna(df3['location']), other = df3['time_zone'])

In [10]:
pd.isna(df3['tweet']).sum()

28459

In [11]:
#drop columns 'text' & 'full_text'
df3 = df3.drop(['text', 'full_text', 'location', 'time_zone'], axis=1)
df3.rename(columns = {'location_':'location'}, inplace = True)
df3.reindex(columns = ['created_at','location','tweet'])

#drop rows all elements are NaN

df3 = df3.dropna(how='all')

In [12]:
#rename locations taken from time_zone ex) Eastern Time (US & Canada) -> US & Canada EastCoast etc 
#df3.to_csv('test2.csv')

In [13]:
#Extract hashtag from 'tweet' and create a new cloumn 'hashtag'
df3['hashtag'] = df3['tweet'].str.findall(r'#.*?\s')

In [14]:
df3.reindex(columns = ['created_at','location','tweet','hashtag'])

,created_at,location,tweet,hashtag
0,2018-03-25 09:12:15,Planet Awesome,rt @nickjonas: so inspired by the #marchforour...,[#marchforourlives ]
1,2018-03-25 09:12:22,Huarte (Navarra),-“somos niños. no queremos que nos maten”. los...,"[#parkland , #marchforourlives#guncontrol ]"
2,2018-03-25 09:12:26,"Portland, OR",my view before speaking at the south bay #marc...,"[#marchforourlives , #hermosabeach , #gunviole..."
3,2018-03-25 09:12:27,"Melbourne, FL",@proudresister your marches made prime time ne...,[#guncontrol ]
4,2018-03-25 09:12:28,Manchester,"dear @davidhogg111, you’re clearly talking to ...",[#guncontrol. ]
...,...,...,...,...
414311,2018-03-25 18:44:25,USA,when you have a mayor &amp; law enforcement th...,"[#wakeupamerica , #marchforourlives , #chicago..."
414312,2018-03-25 18:44:25,"รามอินทรา, ประเทศไทย",#marchforourlives ชาวสหรัฐร่วมกันออกมาแสดงพลัง...,[#marchforourlives ]
414313,2018-03-25 18:44:25,room 93,this is martin palla.he found it necessary to ...,[#marchforourlives ]
414314,2018-03-25 18:44:25,"Washington, DC","""i’m going to start off by putting this price ...",[]


In [15]:
df3['created_at'] = df3['created_at'].apply(lambda x: x.round('min'))

In [16]:
df3.groupby('created_at').count()

,tweet,location,hashtag
created_at,,,
2018-03-25 09:12:00,6,6,6
2018-03-25 09:13:00,11,14,11
2018-03-25 09:14:00,16,20,16
2018-03-25 09:15:00,15,21,15
2018-03-25 09:16:00,18,19,18
...,...,...,...
2018-03-25 18:40:00,1817,1907,1817
2018-03-25 18:41:00,1698,1770,1698
2018-03-25 18:42:00,1907,1984,1907


In [17]:
df3.groupby('created_at').count()['tweet']

created_at
2018-03-25 09:12:00       6
2018-03-25 09:13:00      11
2018-03-25 09:14:00      16
2018-03-25 09:15:00      15
2018-03-25 09:16:00      18
                       ... 
2018-03-25 18:40:00    1817
2018-03-25 18:41:00    1698
2018-03-25 18:42:00    1907
2018-03-25 18:43:00    1820
2018-03-25 18:44:00    1669
Name: tweet, Length: 297, dtype: int64

In [18]:
%matplotlib notebook
df3.groupby('created_at').count()['tweet'].plot()

<IPython.core.display.Javascript object>

In [19]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

In [20]:
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [21]:
tweet = df3['tweet'][10]

In [22]:
print(preprocess(tweet))

['#harshfacts', 'some', 'of', 'you', 'that', 'attended', 'or', 'support', '#marchforourlives', 'probably', 'have', 'a', 'relative', 'or', 'knows', 'someone', 'who', 'is', '…', 'https://t.co/hmeuthorw8']


In [23]:
df4 = df3.dropna(axis=0, inplace=False)
df4

,created_at,tweet,location,hashtag
0,2018-03-25 09:12:00,rt @nickjonas: so inspired by the #marchforour...,Planet Awesome,[#marchforourlives ]
1,2018-03-25 09:12:00,-“somos niños. no queremos que nos maten”. los...,Huarte (Navarra),"[#parkland , #marchforourlives#guncontrol ]"
2,2018-03-25 09:12:00,my view before speaking at the south bay #marc...,"Portland, OR","[#marchforourlives , #hermosabeach , #gunviole..."
3,2018-03-25 09:12:00,@proudresister your marches made prime time ne...,"Melbourne, FL",[#guncontrol ]
4,2018-03-25 09:12:00,"dear @davidhogg111, you’re clearly talking to ...",Manchester,[#guncontrol. ]
...,...,...,...,...
414311,2018-03-25 18:44:00,when you have a mayor &amp; law enforcement th...,USA,"[#wakeupamerica , #marchforourlives , #chicago..."
414312,2018-03-25 18:44:00,#marchforourlives ชาวสหรัฐร่วมกันออกมาแสดงพลัง...,"รามอินทรา, ประเทศไทย",[#marchforourlives ]
414313,2018-03-25 18:44:00,this is martin palla.he found it necessary to ...,room 93,[#marchforourlives ]
414314,2018-03-25 18:44:00,"""i’m going to start off by putting this price ...","Washington, DC",[]


In [24]:
df4['preprocess_tweet'] = df4['tweet'].apply(lambda x : preprocess(x))
#df4['test'] = df4['tweet'].apply(lambda x: type(x))
#df4.to_csv('test3.csv')

/Users/satokosuda/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [25]:
df4

,created_at,tweet,location,hashtag,preprocess_tweet
0,2018-03-25 09:12:00,rt @nickjonas: so inspired by the #marchforour...,Planet Awesome,[#marchforourlives ],"[rt, @nickjonas, :, so, inspired, by, the, #ma..."
1,2018-03-25 09:12:00,-“somos niños. no queremos que nos maten”. los...,Huarte (Navarra),"[#parkland , #marchforourlives#guncontrol ]","[-, “, somos, niños, ., no, queremos, que, nos..."
2,2018-03-25 09:12:00,my view before speaking at the south bay #marc...,"Portland, OR","[#marchforourlives , #hermosabeach , #gunviole...","[my, view, before, speaking, at, the, south, b..."
3,2018-03-25 09:12:00,@proudresister your marches made prime time ne...,"Melbourne, FL",[#guncontrol ],"[@proudresister, your, marches, made, prime, t..."
4,2018-03-25 09:12:00,"dear @davidhogg111, you’re clearly talking to ...",Manchester,[#guncontrol. ],"[dear, @davidhogg111, ,, you, ’, re, clearly, ..."
...,...,...,...,...,...
414311,2018-03-25 18:44:00,when you have a mayor &amp; law enforcement th...,USA,"[#wakeupamerica , #marchforourlives , #chicago...","[when, you, have, a, mayor, &, amp, ;, law, en..."
414312,2018-03-25 18:44:00,#marchforourlives ชาวสหรัฐร่วมกันออกมาแสดงพลัง...,"รามอินทรา, ประเทศไทย",[#marchforourlives ],"[#marchforourlives, ชาวสหร, ั, ฐร, ่, วมก, ั, ..."
414313,2018-03-25 18:44:00,this is martin palla.he found it necessary to ...,room 93,[#marchforourlives ],"[this, is, martin, palla, ., he, found, it, ne..."
414314,2018-03-25 18:44:00,"""i’m going to start off by putting this price ...","Washington, DC",[],"["", i, ’, m, going, to, start, off, by, puttin..."


In [26]:
#nltk.download('stopwords')
from nltk.corpus import stopwords
import string

punc = list(string.punctuation)
stop = stopwords.words('english') + punc + ['rt', 'via', '’', 'amp']

/Users/satokosuda/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/satokosuda/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [27]:
terms_nostop = [term for term in preprocess(tweet) if term not in stop]

In [28]:
# There are 37487 rows in df4. for each item (list) in the 'preprocess_tweet' column, check if it's non-empty, them 
# remove stop words from the item and append it to the longer 'items' list. Since the data is large, will run the loop
# for 4 times (10,000 x 3 + 7487 X 1)
# increase data rate by jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

items = []
for item in df4['preprocess_tweet'][:10000]:
    if len(item) !=0:
        for i in item:
            if i not in stop and not i.startswith(('#', '@')):
                items.append(i.strip())
            else:
                continue  
    else:
        continue
        
print(items[:100])

['inspired', 'posts', 'today', 'keeping', 'japan', 'pray', 'change', 'comes', 'need', 'n', '…', '“', 'somos', 'niños', 'queremos', 'que', 'nos', 'maten', '”', 'los', 'supervivientes', 'de', 'han', 'movilizado', 'al', 'país', 'contra', 'las', 'armas', 'se', 'esperan', 'manifestaciones', 'multitudinarias', 'un', 'día', 'que', 'puede', 'hacer', 'historia', 'https://t.co/xaj0c88cx7', 'view', 'speaking', 'south', 'bay', 'pier', 'thousands', 'americans', 'overflowed', 'pier', 'demand', 'common', 'sense', 'prevention', 'legislation', 'https://t.co/aeo98ltgzf', 'marches', 'made', 'prime', 'time', 'news', 'australia', 'talked', 'strong', 'voice', 'cheering', 'guys', 'dear', 'clearly', 'talking', 'wrong', 'crowd', 'talking', 'hillary', 'obama', 'sent', 'guns', 'weapons', 'isis', 'resulted', 'deaths', 'family', 'people', 'iraq', 'thank', 'morristown', 'nj', '10', 'k', 'people', 'strong', 'https://t.co/gsrj8pm0sp', 'marched', 'lives', 'marched', 'women', 'rights', 'marched', 'science', 'marched', 

In [29]:
import operator
from collections import Counter

count_all = Counter()
count_all.update(items)
print(count_all.most_common(30))    

[('gun', 1574), ('today', 1313), ('thank', 994), ('us', 984), ('people', 935), ('world', 865), ('march', 852), ('kids', 811), ('guns', 803), ('violence', 776), ('children', 739), ('school', 707), ('students', 700), ('support', 675), ('also', 663), ('🏻', 661), ('…', 632), ('️', 628), ('fight', 610), ('lives', 608), ('future', 582), ('control', 582), ('nra', 570), ('“', 553), ('enough', 531), ('weapons', 529), ('good', 527), ('like', 523), ('”', 496), ('👇', 479)]


In [30]:
hashlist = []
for item in df4['hashtag']:
    if len(item) !=0:
        for i in item:
            if i.startswith('#'):
                hashlist.append(i.strip())
            else:
                continue  
    else:
        continue
        
print(hashlist[:100])

['#marchforourlives', '#parkland', '#marchforourlives#guncontrol', '#marchforourlives', '#hermosabeach', '#gunviolence', '#guncontrol', '#guncontrol.', '#marchforourlives', '#enough', '#vote', '#gop', '#enough', '#marchforourlives', '#paris', '#imarchfor', '#marchforourlives', '#theresistance', '#resistance', '#impeachtrump', '#notmypresident', '#resist', '#trump', '#neveragain', '#enough', '#marchforourlives', '#parklandstrong', '#notonemore', '#harshfacts', '#marchforourlives', '#guncontrol', '#schoolsafety', '#marchforourlives', '#gunfreezoneskill', '#2a', '#guncontrol', '#guncontrol.', '#guncontrol.', '#marchforourlives', '#hermosabeach', '#gunviolence', '#guncontrol.', '#neveragain', '#marchforourlives', '#enough', '#wecallbs', '#guncontrolyou', '#enough', '#marchforourlives', '#goptreason#goptraitors', '#gopcomplicit@foxnews', '#marchforourlives', '#hermosabeach', '#gunviolence', '#gunviolence', '#gunshowloophole', '#gunsafetylaws', '#marchforourlives', '#enough', '#davidhogg', '

In [31]:
count_hash = Counter()
count_hash.update(hashlist)
print(count_hash.most_common(30))

[('#marchforourlives', 239892), ('#neveragain', 26211), ('#marchforourlives.', 8029), ('#marchforourlives:', 5850), ('#2a', 5756), ('#nra', 5198), ('#maga', 5136), ('#guncontrolnow', 4946), ('#enough', 4571), ('#msdstrong', 4313), ('#votethemout', 4287), ('#guncontrol', 3612), ('#standup', 3119), ('#uva', 3108), ('#enoughisenough', 3020), ('#gunreform', 2719), ('#veteransforgunreform', 2701), ('#marchforourlives,', 2113), ('#vetsforgunreform', 1965), ('#naomiwadler!', 1877), ('#marchforourlives\\xa0', 1793), ('#marchforourlivessigns', 1783), ('#vetsvsthenra', 1745), ('#gunreformnow', 1740), ('#trump', 1677), ('#sundaymorning', 1616), ('#gunviolence', 1590), ('#marchforourlivessacramento', 1557), ('#resist', 1482), ('#secondamendment', 1453)]


In [32]:
# to remove emojis from tokens

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F500"  # symbols & pictographs
        u"\U0001F520-\U0001F52F"
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)                          
        u"\U0001F910-\U0001F96B" 
        u"\U0001F52B"
        u"\U0001F5E3"             # speech
        u"\U0001F5F3"             # vote 
        u"\U0001F91B-\U0001F939"                   
        u"\U0001F191-\U0001F19A"
        u"\U0001F595"
        u"\U0001F985"
                             "]+", flags = re.UNICODE)

In [33]:
items_noemoji = []
for item in items:
    i = emoji_pattern.sub(r'', item)
    items_noemoji.append(i)
    
print(items_noemoji[:100])

['inspired', 'posts', 'today', 'keeping', 'japan', 'pray', 'change', 'comes', 'need', 'n', '…', '“', 'somos', 'niños', 'queremos', 'que', 'nos', 'maten', '”', 'los', 'supervivientes', 'de', 'han', 'movilizado', 'al', 'país', 'contra', 'las', 'armas', 'se', 'esperan', 'manifestaciones', 'multitudinarias', 'un', 'día', 'que', 'puede', 'hacer', 'historia', 'https://t.co/xaj0c88cx7', 'view', 'speaking', 'south', 'bay', 'pier', 'thousands', 'americans', 'overflowed', 'pier', 'demand', 'common', 'sense', 'prevention', 'legislation', 'https://t.co/aeo98ltgzf', 'marches', 'made', 'prime', 'time', 'news', 'australia', 'talked', 'strong', 'voice', 'cheering', 'guys', 'dear', 'clearly', 'talking', 'wrong', 'crowd', 'talking', 'hillary', 'obama', 'sent', 'guns', 'weapons', 'isis', 'resulted', 'deaths', 'family', 'people', 'iraq', 'thank', 'morristown', 'nj', '10', 'k', 'people', 'strong', 'https://t.co/gsrj8pm0sp', 'marched', 'lives', 'marched', 'women', 'rights', 'marched', 'science', 'marched', 

In [34]:
from nltk import bigrams

terms_bigram = list(bigrams(items_noemoji))
count_bigram = Counter()
count_bigram.update(terms_bigram)
print(count_bigram.most_common(30))

[(('', ''), 1777), (('gun', 'control'), 502), (('gun', 'violence'), 458), (('us', 'tomorrow'), 377), (('tomorrow', 'world'), 374), (('https://t.co/riqlxq55mc', 'https://t.co/tydiv7czca'), 374), (('service', 'standing'), 374), (('thank', 'service'), 374), (('absolutely', 'words'), 374), (('words', 'thank'), 374), (('standing', 'us'), 374), (('world', 'https://t.co/riqlxq55mc'), 374), (('young', 'people'), 284), (('common', 'sense'), 277), (('support', 'fight'), 244), (('vulnerable', 'violence'), 241), (('respect', 'future'), 241), (('animals', 'vulnerable'), 241), (('future', 'activists'), 241), (('violent', 'dog'), 241), (('children', 'animals'), 241), (('dog', 'cat'), 241), (('fight', 'violent'), 241), (('humanity', 'live'), 241), (('deepest', 'respect'), 241), (('trade', 'send'), 241), (('live', 'pls'), 241), (('violence', 'deepest'), 241), (('cat', 'meat'), 241), (('activists', 'may'), 241)]


In [35]:
#import sys
#!conda install --yes --prefix {sys.prefix} vincent

In [36]:
import vincent
vincent.core.initialize_notebook()

hash_freq = count_hash.most_common(30)  #create list of tuples
labels, freq = zip(*hash_freq)  #seperate the above into 1. tuple of labels & 2. tuple of counts
data = {'data': freq, 'x': labels} #create dictionary of tuples
bar = vincent.Bar(data, iter_idx='x')
bar.display()

ImportError: No module named 'vincent'

In [ ]:
beach_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#hermosabeach' in df4['tweet'].iloc[i]:
        beach_hash_time.append(df4['created_at'].iloc[i])
        
print(len(beach_hash_time))
print(beach_hash_time[:100])

In [ ]:
from pandas.tseries.offsets import Hour
one_hour = Hour(1)
one_hour

In [ ]:
# Creating datetimeindex for time series data for pandas. Twitter streaming data is based on UTC time.
# From US & Canada ETS time, it's 4 hours earlier. Adjust for daytime savings.

idx = pd.DatetimeIndex(beach_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(beach_hash_time)
#ones
beach_hash = pd.Series(ones, index=idx_est_ds)
per_minute = beach_hash.resample('1min').sum().fillna(0)

In [ ]:
time_chart = vincent.Line(per_minute)
time_chart.axis_titles(x='Time', y='Hashtag frequencies')
time_chart.display()

In [ ]:
per_minute

In [ ]:
march_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#marchforourlives' in df4['tweet'].iloc[i]:
        march_hash_time.append(df4['created_at'].iloc[i])
        
idx = pd.DatetimeIndex(march_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(march_hash_time)
#ones
march_hash = pd.Series(ones, index=idx_est_ds)
march_per_minute = march_hash.resample('1min').sum().fillna(0)

In [ ]:
enough_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#enough' in df4['tweet'].iloc[i]:
        enough_hash_time.append(df4['created_at'].iloc[i])
        
idx = pd.DatetimeIndex(enough_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(enough_hash_time)
#ones
enough_hash = pd.Series(ones, index=idx_est_ds)
enough_per_minute = enough_hash.resample('1min').sum().fillna(0)

In [ ]:
again_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#neveragain' in df4['tweet'].iloc[i]:
        again_hash_time.append(df4['created_at'].iloc[i])
        
idx = pd.DatetimeIndex(again_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(again_hash_time)
#ones
again_hash = pd.Series(ones, index=idx_est_ds)
again_per_minute = again_hash.resample('1min').sum().fillna(0)

In [ ]:
control_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#guncontrol' in df4['tweet'].iloc[i]:
        control_hash_time.append(df4['created_at'].iloc[i])
        
idx = pd.DatetimeIndex(control_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(control_hash_time)
#ones
control_hash = pd.Series(ones, index=idx_est_ds)
control_per_minute = control_hash.resample('1min').sum().fillna(0)

In [ ]:
violence_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#gunviolence' in df4['tweet'].iloc[i]:
        violence_hash_time.append(df4['created_at'].iloc[i])
        
idx = pd.DatetimeIndex(violence_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(violence_hash_time)

violence_hash = pd.Series(ones, index=idx_est_ds)
violence_per_minute = violence_hash.resample('1min').sum().fillna(0)

In [ ]:
school_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#schoolshooting' in df4['tweet'].iloc[i]:
        school_hash_time.append(df4['created_at'].iloc[i])
        
idx = pd.DatetimeIndex(school_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(school_hash_time)
#ones
school_hash = pd.Series(ones, index=idx_est_ds)
school_per_minute = school_hash.resample('1min').sum().fillna(0)

In [ ]:
time_chart = vincent.Line(school_per_minute)
time_chart.axis_titles(x='Time', y='Hashtag frequencies')
time_chart.display()

In [ ]:
match_data = dict(Beach=per_minute, March=march_per_minute, Enough=enough_per_minute, Again=again_per_minute, Control=control_per_minute, Violence=violence_per_minute)
all_matches = pd.DataFrame(data = match_data,index=march_per_minute.index)
all_matches

In [ ]:
time_chart_top3 = vincent.Line(all_matches[['March', 'Enough', 'Again']])
time_chart_top3.axis_titles(x='Time', y='Freq')
time_chart_top3.legend(title='Top 3 Most popular hashtags')
time_chart_top3.display()

In [ ]:
time_chart_next3 = vincent.Line(all_matches[['Control', 'Violence', 'Beach']])
time_chart_next3.axis_titles(x='Time', y='Freq')
time_chart_next3.legend(title='Top 4 to 6 Most popular hashtags')
time_chart_next3.display()

In [ ]:
pro_guncontrol = []  #collection of words which indictaes the tweet is pro guncontrol
anti_guncontrol = []  #collection of words which indicates the tweet is anti guncontrol

In [ ]:
#df4['states'] = np.where(df4['clean_states'] !="USA", df4['clean_states'], df4['clean_states2'])df4['clean_states'])

#for i in range(len(df4['clean_states'])):
#    if df4['clean_states'].iloc[i][0] == 'USA':
#        df4['states'][i] = df4['clean_states2'][i]
#    else:
#        df4['states'][i] = df4['clean_states'][i]
#df4['clean_states'].iloc[1][0]